In [1]:
%pwd

'/home/ebi/machinelearning/end_to_end_ml_projects/end_to_end_food_vision/research'

In [2]:
%cd ../

/home/ebi/machinelearning/end_to_end_ml_projects/end_to_end_food_vision


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataLoaderConfig:
    root_dir: Path
    train_dir: Path
    test_dir: Path
    image_shape: tuple
    batch_size: int
    class_mode: str

In [6]:
from cnnClassifier.constants import *
import os
from pathlib import Path
from cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH, 
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_loader_config(self) -> DataLoaderConfig:
        config = self.config.data_loader
        params = self.params

        data_loader_config = DataLoaderConfig(
            root_dir=config.root_dir,
            train_dir=config.train_dir, 
            test_dir=config.test_dir, 
            image_shape=params.IMAGE_SHAPE, 
            batch_size=params.BATCH_SIZE, 
            class_mode=params.CLASS_MODE
        )

        return data_loader_config

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class DataLoader:
    def __init__(self, config: DataLoaderConfig):
        self.config = config
    
    def load_data(self):
        self.train_dir = self.config.train_dir
        self.test_dir = self.config.test_dir

    def prepare_data(self):
        train_datagen = ImageDataGenerator(rescale=1/255.)
        test_datagen = ImageDataGenerator(rescale=1/255.)

        train_data_10_percent = train_datagen.flow_from_directory(self.train_dir, 
                                                                  target_size=self.config.image_shape, 
                                                                  batch_size=self.config.batch_size, 
                                                                  class_mode=self.config.class_mode)
        test_data = train_datagen.flow_from_directory(self.test_dir, 
                                                      target_size=self.config.image_shape, 
                                                      batch_size=self.config.batch_size, 
                                                      class_mode=self.config.class_mode)
        return train_data_10_percent, test_data


In [8]:
from cnnClassifier import logger

STAGE_NAME = "Load Data"

try:
    logger.info(f">>>>>>>> stage {STAGE_NAME} started <<<<<<<<<<<<")
    config = ConfigurationManager()
    data_loader_config = config.get_data_loader_config()
    data_loader = DataLoader(config=data_loader_config)
    data_loader.load_data()
    train, test = data_loader.prepare_data()
    logger.info(f">>>>>>>> stage {STAGE_NAME} completed <<<<<<<<<<\n\nx============x")

except Exception as e:
    raise e

[2023-10-09 15:07:28,081: INFO: 750901943: >>>>>>>> stage Load Data started <<<<<<<<<<<<]
[2023-10-09 15:07:28,086: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-09 15:07:28,097: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-09 15:07:28,099: INFO: common: created directory at: artifacts]
Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.
[2023-10-09 15:07:28,313: INFO: 750901943: >>>>>>>> stage Load Data completed <<<<<<<<<<

x============x]
